In [568]:
from os import path
import logging

import pandas as pd

In [569]:
import sys
sys.path.append("../") # go to parent dir

In [570]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder, ModelStorageService

In [571]:
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.INFO)

In [572]:
filename = path.join('..', 'tests', 'resources', 'data', 'compact', 'towns.csv')
data = pd.read_csv(filename, delimiter=';')
properties = ['area', 'country', 'town']
source_df = pd.DataFrame(data, columns=properties)

In [573]:
schema_towns = SchemaBuilder().level('area', 'A').level('country', 'C').level('town', 'T').resource('monument', 'M').render()

model = ModelBuilder().from_compact(source_df) \
                      .with_schema(schema_towns) \
                      .render()

INFO:dependencynet.schema:rendering schema
INFO:dependencynet.model:extract_items_root keys=['area'] id_pattern=A{id:02d} => shape=(2, 4)
INFO:dependencynet.model:extract_items_root keys=['area', 'country'] id_pattern={id_parent}C{id:02d} => shape=(4, 6)
INFO:dependencynet.model:extract_items_root keys=['area', 'country', 'town'] id_pattern={id_parent}T{id:02d} => shape=(5, 7)


In [574]:
import ipycytoscape

In [575]:
import networkx as nx

In [576]:
G = nx.DiGraph()

nodes

In [577]:
class CustomNode(ipycytoscape.Node):
    def __init__(self, id, label, classes):
        super().__init__()
        self.data['id'] = id       
        self.data['label'] = label       
        self.classes = classes

In [578]:
class AreaNode(CustomNode):
    def __init__(self, properties):
        super().__init__(properties['id'], properties['label'], 'area')


In [579]:
class CountryNode(CustomNode):
    def __init__(self, properties):
        super().__init__(properties['id'], properties['label'], 'country')


In [603]:
class GraphBuilder():
    @staticmethod
    def show_graph(G, layout_name, graph_style, rank_dir):
        cytoscapeobj = ipycytoscape.CytoscapeWidget()
        cytoscapeobj.set_style(graph_style)
        cytoscapeobj.graph.add_graph_from_networkx(G)
        
        cytoscapeobj.set_layout(name=layout_name, nodeSpacing=10, edgeLenghVal=10, rankDir=rank_dir)
        # klay, dagre, cola
       
        cytoscapeobj.set_tooltip_source('label')
        
        display(cytoscapeobj)
        
    
    @staticmethod
    def node_from_area(area_df, G):
            wf_records = area_df.to_dict('records')
            wf_nodes = [AreaNode(row) for row in wf_records]
            G.add_nodes_from(wf_nodes)
        
    @staticmethod
    def node_from_country(country_df, G):
            wf_records = country_df.to_dict('records')
            wf_nodes = [CountryNode(row) for row in wf_records]
            G.add_nodes_from(wf_nodes)
        
        

In [581]:
area_df = model.level_dataset(0)
country_df = model.level_dataset(1)
GraphBuilder.node_from_area(area_df, G)
GraphBuilder.node_from_country(country_df, G)

edges

In [582]:
area_nodes_by_id = {n.data['id']:n for n in G.nodes() if 'area' in n.classes}
country_nodes_by_id = {n.data['id']:n for n in G.nodes() if 'country' in n.classes}

In [583]:
def add_area_country_edge(row):
    id_country = row['id']
    id_area = row['id_parent']
    print(f'{id_area} -> {id_country}')
    G.add_edge(area_nodes_by_id[id_area], country_nodes_by_id[id_country], label='country')
    
country_edges = country_df.apply(add_area_country_edge, axis=1)

A01 -> A01C01
A01 -> A01C02
A01 -> A01C03
A02 -> A02C01


In [584]:
graph_style_ex = [ 
    {
        "selector" : "node",
        "css" : {
          "text-valign" : "center",
          "text-halign" : "center",
          "font-family" : "SansSerif.plain",
          "font-weight" : "normal",
          "background-color" : "rgb(204,255,255)",
          "border-width" : 1.5,
          "height" : 35.0,
          "width" : 75.0,
          "border-opacity" : 1.0,
          "color" : "rgb(0,0,0)",
          "border-color" : "rgb(0,153,255)",
          "text-opacity" : 1.0,
          "background-opacity" : 1.0,
          "font-size" : 12,
          "shape" : "ellipse",
          "content" : "data(src_node)"
        }
    }
]

In [602]:
graph_style = [ 
    {
        'selector' : 'node.area',
        'css' : {
            'shape' : 'roundrectangle',
            'label' : 'data(id)',
            'content' : 'data(label)',
            'font-size' : '4pt',
            'text-valign' : 'center',
            'text-halign' : 'center',
            'color' : 'white',
            'background-color' : 'black',
            'height' : '0.8em',
            'width' : '4em',
            'text-wrap': 'wrap',
            'text-max-width' : '5em' 
           }
    }, 
    {
         'selector' : 'node.country',
        'css' : {
            'shape' : 'roundrectangle',
            'label' : 'data(id)',
            'content' : 'data(label)',
            'font-size' : '4pt',
            'text-valign' : 'center',
            'text-halign' : 'center',
            'color' : 'white',
            'background-color' : 'black',
            'height' : '0.8em',
            'width' : '6em',
            'text-wrap': 'wrap',
            'text-max-width' : '7em' 
       }
    }, 
    {
         'selector' : 'edge',
         'css' : {
            'shape' : 'arrow',
            'width' : 1,
            'line-color' : 'gray',
            'curve-style' : 'bezier',
            'target-arrow-color' : 'gray',
            'target-arrow-shape' : 'triangle'
       }
    }
]

In [586]:
print(G.nodes)

[AreaNode(classes='area', data={'id': 'A01', 'label': 'A01 Europe'}, position={}), AreaNode(classes='area', data={'id': 'A02', 'label': 'A02 Asia'}, position={}), CountryNode(classes='country', data={'id': 'A01C01', 'label': 'A01C01 France'}, position={}), CountryNode(classes='country', data={'id': 'A01C02', 'label': 'A01C02 UK'}, position={}), CountryNode(classes='country', data={'id': 'A01C03', 'label': 'A01C03 Italia'}, position={}), CountryNode(classes='country', data={'id': 'A02C01', 'label': 'A02C01 Japan'}, position={})]


In [587]:
import matplotlib.pyplot as plt
%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


dagre klay

In [605]:
GraphBuilder.show_graph(G, 'dagre', graph_style, 'LR')

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cy…

In [589]:
import pyyed

Valid node shapes are: "rectangle", "rectangle3d", "roundrectangle", "diamond", "ellipse", "fatarrow", "fatarrow2", "hexagon", "octagon", "parallelogram", "parallelogram2", "star5", "star6", "star6", "star8", "trapezoid", "trapezoid2", "triangle", "trapezoid2", "triangle"

Valid line_types are: "line", "dashed", "dotted", "dashed_dotted"

Valid font_styles are: "plain", "bold", "italic", "bolditalic"

Valid arrow_types are: "none", "standard", "white_delta", "diamond", "white_diamond", "short", "plain", "concave", "concave", "convex", "circle", "transparent_circle", "dash", "skewed_dash", "t_shape", "crows_foot_one_mandatory", "crows_foot_many_mandatory", "crows_foot_many_optional", "crows_foot_many_optional", "crows_foot_one", "crows_foot_many", "crows_foot_optional"

custom properties 
https://github.com/jamesscottbrown/pyyed/blob/master/examples/demo-custom-properties-nodes-edges.py

In [590]:
display(*G)
for node in G.nodes(data=True):
    print(node[0])

AreaNode(classes='area', data={'id': 'A01', 'label': 'A01 Europe'}, position={})

AreaNode(classes='area', data={'id': 'A02', 'label': 'A02 Asia'}, position={})

CountryNode(classes='country', data={'id': 'A01C01', 'label': 'A01C01 France'}, position={})

CountryNode(classes='country', data={'id': 'A01C02', 'label': 'A01C02 UK'}, position={})

CountryNode(classes='country', data={'id': 'A01C03', 'label': 'A01C03 Italia'}, position={})

CountryNode(classes='country', data={'id': 'A02C01', 'label': 'A02C01 Japan'}, position={})

AreaNode(classes='area', data={'id': 'A01', 'label': 'A01 Europe'}, position={})
AreaNode(classes='area', data={'id': 'A02', 'label': 'A02 Asia'}, position={})
CountryNode(classes='country', data={'id': 'A01C01', 'label': 'A01C01 France'}, position={})
CountryNode(classes='country', data={'id': 'A01C02', 'label': 'A01C02 UK'}, position={})
CountryNode(classes='country', data={'id': 'A01C03', 'label': 'A01C03 Italia'}, position={})
CountryNode(classes='country', data={'id': 'A02C01', 'label': 'A02C01 Japan'}, position={})


In [591]:
import pyyed
import codecs

class YedBuilder:
    @staticmethod
    def build(G):
        g = pyyed.Graph()
        
        colors = { 'area': '#FFFF88', 'country': '#FFFFCC'}
        shapes = { 'area': 'roundrectangle', 'country': 'roundrectangle'}
        widths = { 'area': '80', 'country': '120'}
       
        for node in G.nodes(data=True):
            n = node[0]
            g.add_node(n.data["id"], label=n.data["label"],
                                     shape=shapes[n.classes], 
                                     shape_fill=colors[n.classes],
                                     font_size='8',
                                     width=widths[n.classes],
                                     height='30')
    
        for n1, n2 in G.edges():
            g.add_edge(n1.data["id"], n2.data["id"], arrowhead="standard", arrowfoot="none")

        filename = path.join('output', 'testyed.graphml')
 
        with codecs.open(filename, 'w', 'utf-8') as fp:
            graph = g.get_graph()
            fp.write(graph)
            print(filename)


In [592]:
YedBuilder.build(G)

output/testyed.graphml


In [593]:
class YedBuilderTest:
    @staticmethod
    def build(G):
        g = pyyed.Graph()

        n1 = g.add_node("A", label="A", shape="roundrectangle", shape_fill="#FFFFFF")
        n2 = g.add_node("B", label="B", shape="roundrectangle", shape_fill="#FFFFFF")
        n3 = g.add_node("C", label="C", shape="roundrectangle", shape_fill="#FFFFFF")
        n4 = g.add_node("D", label="D", shape="roundrectangle", shape_fill="#FFFFFF")
        n5 = g.add_node("E", label="E", shape="roundrectangle", shape_fill="#FFFFFF")

        g.add_edge("A", "C", arrowhead="standard", arrowfoot="none")
        g.add_edge("B", "C", arrowhead="standard", arrowfoot="none")
        g.add_edge("C", "D", arrowhead="standard", arrowfoot="none")
        g.add_edge("C", "E", arrowhead="standard", arrowfoot="none")

        #g.add_node('abc', font_size="72", height="100", shape_fill="#FFFFFF")

        #g.add_edge('foo', 'foo1', label="EDGE!", width="3.0", color="#0000FF", 
        #               arrowhead="white_diamond", arrowfoot="standard", line_type="dotted")

        # file content
        #print(g.get_graph())

        filename = path.join('output', 'testyed.graphml')
        # To write to file:
        #with open('test_graph.graphml', 'w') as fp:
        #    fp.write(g.get_graph())

        # Or:
        #g.write_graph('example.graphml')

        # Or, to pretty-print with whitespace:
        g.write_graph(filename, pretty_print=True)
        print(filename)
        


In [594]:
YedBuilderTest.build(G)

output/testyed.graphml


In [595]:
pygraphml

<module 'pygraphml' from '/Users/cfalguiere/anaconda3/envs/dependencynet/lib/python3.8/site-packages/pygraphml/__init__.py'>

In [596]:
import pygraphml as gml
import tempfile

In [597]:
class GraphMLBuilderTest:
    @staticmethod
    def build(G):
        g = gml.Graph()
        
        n1 = g.add_node("A")
        n2 = g.add_node("B")
        n3 = g.add_node("C")
        n4 = g.add_node("D")
        n5 = g.add_node("E")

        # Add attribute
        n1['color'] = 'red'

        g.add_edge(n1, n3)
        g.add_edge(n2, n3)
        g.add_edge(n3, n4)
        g.add_edge(n3, n5)

        filename = path.join('output', 'testpygraphml.graphml')
        #fname = tempfile.mktemp()
        parser = gml.GraphMLParser()
        #parser.write(g, fname)
        parser.write(g, filename)
        print(filename)
        
        # back - parser
        # parser = GraphMLParser()
        # g = parser.parse(fname)

        # g.show()
        # Read attribute
        # print(n['color'])


In [598]:
GraphMLBuilder.build(G)

output/testpygraphml.graphml


In [599]:
from bs4 import BeautifulSoup

fp = "output/testpygraphml.graphml"

with open(fp) as file:
    soup = BeautifulSoup(file, "lxml")

    nodes = soup.findAll("node", {"yfiles.foldertype":""})
    groups = soup.find_all("node", {"yfiles.foldertype":"group"})
    edges = soup.findAll("edge")

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
for node in G.nodes(data=True):
    print node

In [ ]:
from ipywidgets import Output
from IPython.display import display
from pprint import pformat

out = Output()

def log_clicks(node):
    with out:
        print(f'clicked: {pformat(node)}')

def log_mouseovers(node):
    with out:
        print(f'mouseover: {pformat(node)}')

cyto.on('node', 'click', log_clicks)
cyto.on('node', 'mouseover', log_mouseovers)
